In [ ]:
from SetUp import *

postgres = PostgresSetup()

# Query 1

In [ ]:
postgres.query('queries/q1.sql', db_name='yelp', file=True, explain=False, output=False)

In [ ]:
postgres.query("""
SELECT * FROM restaurant_businesses_mv;
""",
db_name='yelp', file=False, explain=False, output=True)

,business_id,name,stars
0,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4.5
1,--7PUidqRWpRSpXebiyxTg,Humpty's Family Restaurant,2.0
2,--SJXpAa0E-GCp2smaHf0A,Winn Dixie,2.5
3,--zb12mw2YK-7j6UaHzm8w,Billy Wright's Manuel's Style Hot Tamales,4.5
4,-_1ctmwdtSpjfpFu0pHXzw,Fat Boy’s Pizza,4.5
...,...,...,...
64572,zzXRdzrVhfNWPHD2MeyWeA,Royal Farms,2.0
64573,ZZYV8hA0Ty93TeDKW4_bBw,Coach's BBQ,3.5
64574,ZZyVnWiMjySRcFOTmpMS3w,Perfetto's Pizza,3.0
64575,zzyx5x0Z7xXWWvWnZFuxlQ,Walnut Street Pizza,2.5


In [ ]:
postgres.query("""
EXPLAIN ANALYZE
WITH businesses_categories AS (
    SELECT business_id, name, TRIM(split_category) AS category
    FROM businesses b
    CROSS JOIN LATERAL UNNEST(categories) AS unnest_category
    CROSS JOIN LATERAL REGEXP_SPLIT_TO_TABLE(unnest_category, ',') AS split_category
), restaurant_businesses AS (
    SELECT DISTINCT business_id
    FROM businesses_categories
    WHERE category = 'Restaurants' OR category = 'Food'
)
SELECT business_id, name, stars
FROM businesses
WHERE business_id IN (SELECT business_id FROM restaurant_businesses);
""",
db_name='yelp', file=False, explain=True, output=True)

[('Hash Join  (cost=41947.90..87553.34 rows=150340 width=51) (actual time=340.757..935.730 rows=64577 loops=1)',),
 ('  Hash Cond: ((b.business_id)::text = (businesses.business_id)::text)',),
 ('  ->  Unique  (cost=23554.25..65534.03 rows=150340 width=23) (actual time=285.685..844.787 rows=64577 loops=1)',),
 ('        ->  Nested Loop  (cost=23554.25..61775.53 rows=1503400 width=23) (actual time=285.685..835.874 rows=80049 loops=1)',),
 ('              ->  Gather Merge  (cost=23554.24..41063.81 rows=150340 width=55) (actual time=285.625..412.318 rows=150346 loops=1)',),
 ('                    Workers Planned: 2',),
 ('                    Workers Launched: 2',),
 ('                    ->  Sort  (cost=22554.21..22710.82 rows=62642 width=55) (actual time=225.336..246.415 rows=50115 loops=3)',),
 ('                          Sort Key: b.business_id',),
 ('                          Sort Method: external merge  Disk: 5960kB',),
 ('                          Worker 0:  Sort Method: external mer

# Query 2

In [ ]:
postgres.query('../queries/q2.sql', db_name='yelp', file=True, explain=False, output=False)

In [ ]:
postgres.query("SELECT * FROM elite_reviews_mv", db_name='yelp', file=False, explain=False, output=True)

,review_id,user_id,business_id,elite_users_rating
0,VeLUA1ryxzQMDpkeWgc93w,ouODopBKF3AqfCkuQEnrDg,yfgGVwoWMEm3mWf9iUZ6jQ,4
1,c_q_NPPxyRFPWkVmOKDbeg,kB4jZscAdNAR7sg8Xs0icA,B10mqANgHvL8gBteo1UhAA,4
2,in0pHjItjTQurmLI7vfcDQ,kaa6i7IHMyYIom7DIul9EQ,u7_3L1NBWgxhBM_B-cmmnA,4
3,rOvS4kJiqzy5t1Rx_266rA,2botYYVjkOyNXYhzYZUrIQ,P_x9fEz8efemqOIV46S0tg,4
4,ys8lGUinqqNRIJ3YR39_Xw,zWK0IkGJ4C7tc2NvCbxucQ,j8JOZvfeHEfUWq3gEz6ABQ,5
...,...,...,...,...
1362150,uyVtfn-1e7hw1JI-nofGtA,UZOm2svtUK-lGt4oTvh4LA,8j5bQ0nDwoGiFaTQDNCQTg,5
1362151,v70vWS-r8yvqlrtwamUmWg,dplh43OlE_Rime4bFXLT6A,vb2XWr50QhBpcOtpzxmKWA,5
1362152,F03-EH1AuqjWBdnIzfRL9w,y6v0WKg8pcQAYMjYJE1cxg,54oLdsGU9B8HxFTqCCqV-A,3
1362153,PFdQZ0-P4olxRiX1LIuvUw,xGS_mlHcCUipSKXrHk6CGw,GjEDg2B1u2ohBMJ0JXD-MA,5


In [ ]:
postgres.query("""
EXPLAIN ANALYZE
WITH elite_year AS (
    SELECT user_id, TRIM(split_elites) AS unnest_year
    FROM users u
    CROSS JOIN LATERAL UNNEST(elite) AS unnest_elite
    CROSS JOIN LATERAL REGEXP_SPLIT_TO_TABLE(unnest_elite, ',') AS split_elites
), elite_count AS (
    SELECT user_id, COUNT(*)
    FROM elite_year
    WHERE LENGTH(unnest_year) >= 2
    GROUP BY user_id
), elite_users AS (
    SELECT user_id
    FROM users
    WHERE user_id in (SELECT user_id FROM elite_count)
)

SELECT r.review_id, r.user_id, r.business_id, r.stars AS elite_users_rating
FROM reviews r
WHERE r.user_id IN (SELECT user_id FROM elite_users)
AND r.business_id IN (SELECT business_id FROM restaurant_businesses_mv);
""", db_name='yelp', explain=True)

[('Hash Semi Join  (cost=5364946.13..6390375.75 rows=6994442 width=73) (actual time=9649.384..14928.698 rows=1362155 loops=1)',),
 ('  Hash Cond: ((r.business_id)::text = (restaurant_businesses_mv.business_id)::text)',),
 ('  ->  Hash Semi Join  (cost=5362853.14..6292109.19 rows=6994442 width=73) (actual time=9636.069..14496.711 rows=1725658 loops=1)',),
 ('        Hash Cond: ((r.user_id)::text = (users.user_id)::text)',),
 ('        ->  Seq Scan on reviews r  (cost=0.00..629301.42 rows=6994442 width=73) (actual time=0.036..2203.831 rows=6990247 loops=1)',),
 ('        ->  Hash  (cost=5320600.83..5320600.83 rows=1984665 width=46) (actual time=9635.658..9635.722 rows=91198 loops=1)',),
 ('              Buckets: 131072  Batches: 32  Memory Usage: 1243kB',),
 ('              ->  Hash Join  (cost=151158.43..5320600.83 rows=1984665 width=46) (actual time=1068.236..9607.870 rows=91198 loops=1)',),
 ('                    Hash Cond: ((elite_count.user_id)::text = (users.user_id)::text)',),
 ('

# Query 3

In [ ]:
postgres.query('../queries/q3.sql', db_name='yelp', file=True, output=False)

In [ ]:
postgres.query("""
SELECT *
FROM michelin_guide_mv;
""", db_name='yelp')

,business_name,business_id,total_elite_reviews,elite_users_avg_rating,all_users_avg_rating,michelin_star
0,Tumerico,DVBJRvnCpkqaYl6nHroaMg,147,4.850,5.0,3
1,TeeJay's Sweet Tooth,OwWt11-2Xps3haab2ZpvLQ,113,4.850,5.0,3
2,Smiling With Hope Pizza,OR7VJQ3Nk1wCcIbPN4TCQQ,122,4.844,5.0,3
3,Amelia’s,zuDXSlqm2veuwTD_Kl-pkw,129,4.829,5.0,3
4,Nicey Treat,JeSIzQ1MbaYGMrGDtEAutQ,136,4.824,5.0,3
...,...,...,...,...,...,...
1922,Iron Hill Brewery & Restaurant,5KS5b5-ip8znhJS_CYdzeQ,117,3.684,4.0,1
1923,Pearl's Oyster Bar,QtST1igZAi0q9LxJr6srIQ,109,3.670,4.0,1
1924,Nom Nom Ramen,pm1bStJuol5XmxE_atZhCQ,204,3.647,4.0,1
1925,Lorenzo & Sons Pizza,FD7gMovIYQmhdn23Iq91zQ,295,3.563,4.0,1


In [ ]:
postgres.query("""
EXPLAIN ANALYZE
WITH elite_users_ratings AS (
    SELECT business_id,
    AVG(elite_users_rating) AS elite_users_avg_rating,
    COUNT(*) AS total_elite_reviews
    FROM elite_reviews_mv
    GROUP BY business_id
), elite_businesses_ratings AS (
    SELECT name AS business_name, business_id,
    total_elite_reviews,
    ROUND(elite_users_avg_rating, 3) AS elite_users_avg_rating,
    stars AS all_users_avg_rating
    FROM restaurant_businesses_mv rbmv
    NATURAL JOIN elite_users_ratings eur
)

SELECT *,
CASE 
    WHEN elite_users_avg_rating >= 4.5 AND elite_users_avg_rating <= 5 THEN 3
    WHEN elite_users_avg_rating >= 4 AND elite_users_avg_rating < 4.5 THEN 2
    ELSE 1 END AS Michelin_star
FROM elite_businesses_ratings
WHERE total_elite_reviews >= 100 AND all_users_avg_rating >= 3.8
ORDER BY elite_users_avg_rating DESC;
""", db_name='yelp', explain=True)

[('Sort  (cost=30699.49..30699.57 rows=31 width=92) (actual time=497.456..497.544 rows=1927 loops=1)',),
 ('  Sort Key: (round(eur.elite_users_avg_rating, 3)) DESC',),
 ('  Sort Method: quicksort  Memory: 223kB',),
 ('  ->  Hash Join  (cost=29172.23..30698.72 rows=31 width=92) (actual time=488.015..496.751 rows=1927 loops=1)',),
 ('        Hash Cond: ((rbmv.business_id)::text = (eur.business_id)::text)',),
 ('        ->  Seq Scan on restaurant_businesses_mv rbmv  (cost=0.00..1447.21 rows=29936 width=48) (actual time=0.013..5.583 rows=30008 loops=1)',),
 ("              Filter: (stars >= '3.8'::double precision)",),
 ('              Rows Removed by Filter: 34569',),
 ('        ->  Hash  (cost=29171.39..29171.39 rows=67 width=102) (actual time=487.969..488.004 rows=2550 loops=1)',),
 ('              Buckets: 4096 (originally 1024)  Batches: 1 (originally 1)  Memory Usage: 221kB',),
 ('              ->  Subquery Scan on eur  (cost=29118.05..29171.39 rows=67 width=102) (actual time=381.957